# week 10

## goals:
- 

## todo
- 


## notes:

looked at:
- Pacific Islands (PIPA)
- Palau National Marine Sanctuary
- Natural Park of the Coral Seas


- Ross Sea
- PIPA
- PMNM
- south/central pacific ocean
- pick out the 5 from Tim White's paper:
    - PIPA - 2010, expanded 2014 <-- some actual results here
    - Pacific Remote Islands - 2009, expanded 2014 (not enough effort)
    - PMNM - yes but low fishing effort (3258 hours, 2016)
    - Pitcairn Islands - yes but low (101 hours, 2016)
    - Nazca-Desventuradas - yes but low (68 hours, 2016)
    
Actual good ones:
- Natural Park of the Coral Seas, 2014 555577562, further off the east coast of Australia
- Marae Moana/Cook Islands, 2017 555624907 - maybe the best one - middle of the Pacific
- French Austral Lands and Seas, 2019 555697868 - south-east of Africa
- Arquipélago Submarino Do Meteor, 2016 555514087 - Atlantic ocean Y shaped area
- Coral Sea, 2018 555556875 just off the east coast of Australia
- Pacífico Mexicano Profundo, 2018 555624307 southwest Mexican coast
- Palau National Marine Sanctuary, 2015, 555622118 oceania (near indonesia, phillippines) 


A bunch of these at the top are coastal around Spain/France:
- Espacio marino del Delta de l'Ebre-Illes Columbretes 2014
- Espacio marino de la Costa da Morte 2014
- Pertuis charentais - Rochebonne 2019




In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
cd /Users/brendan/Masters/gfw_research

/Users/brendan/Masters/gfw_research


In [4]:
import numpy as np
import pandas
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pywdpa
import geopandas
import contextily as ctx
from shapely import geometry
import pretty_html_table

import util

pandas.set_option('display.max_columns', None)
# pandas.set_option('display.max_rows', None)

In [5]:
def load_year(year):
    year = str(year)
    print(f'\r reading year {year}', end='')
    return pandas.read_parquet("data/" + year + ".parquet")

In [6]:
points_by_year = {
    year : load_year(year)
    for year in range(2012, 2021)
}


 reading year 2020

In [7]:
#mpas = util.load_mpas()
mpas = geopandas.read_parquet("data/mpas.parquet")

In [8]:
# sample the points here before converting to geo
# use the sampled points to find the mmsi that are interesting
# only then should we get all of the points of relevent mmsi

SAMPLE_RATIO = 100
points_sampled = []
for year, points in points_by_year.items():
    print(f'\rsampling {str(year)}... ', end='')
    points_sampled.append(points.sample(frac=1/SAMPLE_RATIO))
print('done.')
points_sampled = pandas.concat(points_sampled)
print("converting to geo...  ", end='')
geopoints_sampled = util.convert_to_geo(points_sampled, box=True)
print("done.")


sampling 2020... done.
converting to geo...  done.


In [ ]:
coral_park = mpas[mpas['WDPAID'] == 555577562.0]
coral_park_date = '2014-05-28'
coral_park_results = util.analyze_mpa(geopoints_sampled, points_by_year, coral_park, coral_park_date)
# coral_park_results[0]

In [ ]:
marae_moana = mpas[mpas['WDPA_PID'] == '555624907_A']
marae_moana_results = util.analyze_mpa(geopoints_sampled, points_by_year, marae_moana, '2017-07-13')

In [ ]:
pipa = mpas[mpas['WDPAID'] == 555512002.0]
pipa_date = '2015-01-01'
pipa_results = util.analyze_mpa(geopoints_sampled, points_by_year, pipa, pipa_date, plot_pre_post=False)
# pipa_results[0]

In [ ]:
_, _, _, affected_pipa_points = pipa_results
daily_effort = affected_pipa_points.groupby(['date'], as_index=False).aggregate({'fishing_hours': 'sum'})

In [ ]:
plt.plot(daily_effort['date'], daily_effort['fishing_hours'])

In [ ]:
table = pipa_results[0]
mmsi_of_interest = table[table['in_pre'] > 1000]['mmsi']
points_of_interest = affected_pipa_points[affected_pipa_points['mmsi'].isin(mmsi_of_interest)]

In [ ]:
daily_effort = points_of_interest.groupby(['date'], as_index=False).aggregate({'fishing_hours': 'sum'})

In [ ]:
plt.plot(daily_effort['date'], daily_effort['fishing_hours'])

In [ ]:
x = daily_effort['date']
y = daily_effort['fishing_hours']

In [ ]:
window = 30
z = np.zeros(len(y) - window)
for i in range(len(y) - window):
    z[i] = sum(y[i:i+window])/window

In [ ]:
plt.plot(x[:-window], z)

In [ ]:
baseline_effort = points_sampled.groupby(['date'], as_index=False).aggregate({'fishing_hours': 'sum'})

In [ ]:
def plot_effort(effort, window=30):
    x = effort['date']
    y = effort['fishing_hours']
    z = np.zeros(len(y) - window)
    for i in range(len(y) - window):
        z[i] = sum(y[i:i+window])/window
    plt.plot(x[:-window], z)

In [ ]:
plot_effort(baseline_effort)

In [ ]:
marae_moana = mpas[mpas['WDPA_PID'] == '555624907_A']
marae_moana_points = util.points_of_interest(geopoints_sampled, points_by_year, marae_moana, '2017-07-13')
daily_marae_effort = marae_moana_points.groupby(['date'], as_index=False).aggregate({'fishing_hours': 'sum'})

In [ ]:
plot_effort(daily_marae_effort)